In [1]:
import pandas as pd
from pathlib import Path

In [12]:
path = Path('../data/raw')

files = [f.name for f in path.iterdir() if f.name.endswith('.csv')]

file_df = pd.DataFrame({'file': files})

file_df['date'] = file_df.file.str[:10]

file_df['previous_date'] = file_df.date.apply(lambda f: str((pd.to_datetime(f) - pd.Timedelta('7 days')).date()))

In [15]:
file_df

,file,date,previous_date
0,2021-08-05_variants_of_concern_sample.csv,2021-08-05,2021-07-29
1,2021-08-12_variants_of_concern_sample.csv,2021-08-12,2021-08-05
2,2021-08-19_variants_of_concern_sample.csv,2021-08-19,2021-08-12
3,2021-08-26_variants_of_concern_sample.csv,2021-08-26,2021-08-19
4,2021-09-02_variants_of_concern_sample.csv,2021-09-02,2021-08-26
5,2021-09-09_variants_of_concern_sample.csv,2021-09-09,2021-09-02


In [17]:
for i, row in file_df.iterrows():
    df = pd.read_csv(path/row.file, sep=';')
    
    df_previous = pd.read_csv(f'../data/archive/{row.previous_date}_variants_of_concern_sample.csv')

    df_previous = df_previous[[c for c in df_previous.columns if ('_count' not in c and 'total' not in c)]]
    
    df_complete = df_previous[df_previous.week < df.week.min()].append(df)
    df_complete.to_csv(f'../data/archive/{row.date}_variants_of_concern_sample.csv', index=False)